In [ ]:
from os import path

# Third-party
import astropy.coordinates as coord
from astropy.table import Table, vstack
from astropy.io import fits
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

from pyia import GaiaData

import gala.coordinates as gc
import gala.dynamics as gd
from scipy.stats import binned_statistic
from scipy.special import logsumexp

import emcee

In [ ]:
g = GaiaData('../data/gd1-with-masks.fits')
stream = g[g.pm_mask & g.gi_cmd_mask]

In [ ]:
phi1 = stream.phi1
phi2 = stream.phi2
pm1 = stream.pm_phi1_cosphi2
pm2 = stream.pm_phi2

---

## Significance of spur

In [ ]:
phi1_lims = [(-36, -30), # spur
             (-18, -12), # blob
             (-51, -45)] # control

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 4))

ax.plot(phi1, phi2, 
        marker='o', linewidth=0, ms=3,
        linestyle='none', alpha=0.6, c='k')

ax.set_xlim(-58, -5)
ax.set_ylim(-4, 4)

for lims in phi1_lims:
    ax.axvspan(lims[0], lims[1], zorder=-100, alpha=0.2)

# ax.set_aspect('equal')

In [ ]:
bins = np.arange(-10, 5+1e-3, 0.2)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for i in range(3):
    phi1_mask = (phi1.value > phi1_lims[i][0]) & (phi1.value < phi1_lims[i][1])
    axes[i].hist(phi2[phi1_mask],
                 bins=bins, alpha=1);
    
    print(phi1_mask.sum())

In [ ]:
def lnnormal(x, mu, std):
    return -0.5 * (x-mu)**2 / std**2 - 0.5*np.log(2*np.pi) - np.log(std)

def lnprior(p):
    a_s1, a_s2, a_f, mu_s, lnstd_s1, lnstd_s2, mu_f, lnstd_f = p
    
    lp = 0
    
    fs = [a_f, a_s1, a_s2]
    for f in fs:
        if f < 0 or f > 1:
            return -np.inf
    
    if a_s2 > a_s1 or a_f > (a_s1+a_s2):
        return -np.inf
    
    if sum(fs) > 1:
        return -np.inf
    
    if mu_f < -4 or mu_f > 4 or mu_s < -4 or mu_s > 4:
        return -np.inf
    
    lp += lnnormal(lnstd_f, -1, 5)
    lp += lnnormal(lnstd_s1, -1, 5)
    lp += lnnormal(lnstd_s2, -1, 5)
    
    return lp

def lnlike(p, phi2):
    a_s1, a_s2, a_f, mu_s, lnstd_s1, lnstd_s2, mu_f, lnstd_f = p
    a_bg = 1 - a_f - a_s1 - a_s2
    
    gd1 = lnnormal(phi2, mu_s, np.exp(lnstd_s1))
    gd1_2 = lnnormal(phi2, mu_s, np.exp(lnstd_s2))
    spur = lnnormal(phi2, mu_f, np.exp(lnstd_f))
    bg = np.full_like(gd1, -np.log(5 - -5))
    
    return logsumexp([gd1, gd1_2, spur, bg], b=np.array([a_s1, a_s2, a_f, a_bg])[:, None], axis=0)

def lnprob(p, phi2):
    lp = lnprior(p)
    if not np.all(np.isfinite(lp)):
        return -np.inf
    
    ll = lnlike(p, phi2)
    if not np.all(np.isfinite(ll)):
        return -np.inf
    
    return ll.sum() + lp

In [ ]:
bins = np.arange(-5, 5+1e-3, 0.33)

p0s = []
p0s.append([0.5, 0.1, 0.1, # a_s1, a_s2, a_f
            0.1, np.log(0.2), np.log(0.3), # mu_s, lnstd_s1, lnstd_s2,
            1.25, np.log(0.15)]) # mu_f, lnstd_f

p0s.append([0.3, 0., 0.2, 
            -0.1, np.log(0.1), np.log(0.4),
            -0.6, np.log(0.6)])

p0s.append([0.5, 0.1, 0.1,
            -0.1, np.log(0.25), np.log(0.3), 
            1.25, np.log(0.15)]) 
    
phi2_grid = np.linspace(-5, 5, 256)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for i in range(3):
    phi1_mask = ((phi1.value > phi1_lims[i][0]) & 
                 (phi1.value < phi1_lims[i][1]) &
                 (phi2.value > -5) & (phi2.value < 5))
    axes[i].hist(phi2[phi1_mask],
                 bins=bins, alpha=1, normed=True);
    axes[i].plot(phi2_grid, np.exp(lnlike(p0s[i], phi2_grid)))

In [ ]:
nwalkers = 64
nsteps = 4096
all_samplers = []

for i in range(3):
    p0 = emcee.utils.sample_ball(p0s[i], [1e-3, 1e-2, 1e-2,
                                          1e-1, 1e-3,
                                          1e-1, 1e-3,
                                          1e-3], nwalkers)
    # p0[:, :3] = np.abs(p0[:, :3])
    
    phi1_mask = ((phi1.value > phi1_lims[i][0]) & 
                 (phi1.value < phi1_lims[i][1]) &
                 (phi2.value > -5) & (phi2.value < 5))
    sampler = emcee.EnsembleSampler(nwalkers, p0.shape[1], 
                                    lnpostfn=lnprob, args=(phi2[phi1_mask].value, ))
    _ = sampler.run_mcmc(p0, nsteps)
    
    all_samplers.append(sampler)

In [ ]:
names = [r'$\alpha_{s, 1}$', r'$\alpha_{s, 2}$', r'$\alpha_f$', 
         r'$\mu_s$', r'$\ln\sigma_{s, 1}$', r'$\ln\sigma_{s, 2}$',
         r'$\mu_f$', r'$\ln\sigma_f$']

fig, axes = plt.subplots(sampler.dim, 3, 
                         figsize=(12, sampler.dim*3), 
                         sharex=True, sharey='row')

for i in range(len(all_samplers)):
    sampler = all_samplers[i]
    
    for k in range(sampler.dim):
        for walker in sampler.chain[..., k]:
            axes[k, i].plot(walker, marker='', drawstyle='steps-mid', 
                            color='k', alpha=0.2)

for i in range(len(names)):
    axes[i, 0].set_ylabel(names[i])
            
fig.tight_layout()

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for i in range(3):
    phi1_mask = ((phi1.value > phi1_lims[i][0]) & 
                 (phi1.value < phi1_lims[i][1]) &
                 (phi2.value > -5) & (phi2.value < 5))
    
    axes[i].hist(phi2[phi1_mask],
                 bins=bins, alpha=1, normed=True);
    
    sampler = all_samplers[i]
    idx = sampler.lnprobability[:, -1].argmax()
    show_p = sampler.chain[idx, -1]

    axes[i].plot(phi2_grid, np.exp(lnlike(show_p, phi2_grid)),
                 lw=2, marker='', color='k', zorder=100)


In [ ]:
a_bins = np.linspace(0, 1, 64)

fig, axes = plt.subplots(1, 3, figsize=(15, 4), 
                         sharex=True)

for i in range(3):
    ax = axes[i]
    
    sampler = all_samplers[i]
    flatchain = np.vstack(sampler.chain[:, 2048::16])
    
    a_s1, a_s2, a_f, *_ = flatchain.T
    a_bg = 1 - (a_s1+a_s2+a_f)
    fs = [a_s1+a_s2, a_f, a_bg]
    names = ['gd1', 'feature', 'bg']
    
    print(np.percentile(a_f, [1, 99, 15, 85]))

    for f, name in zip(fs, names):
        ax.hist(f, bins=a_bins, histtype='step', 
                label=name, lw=2)
    
    if i == 0:
        ax.legend(fontsize=16)
    ax.set_xlabel('$f$')

# Significance figure

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(12, 7),
                         sharex='row', constrained_layout=True)

titles = ['Spur', 'Blob', 'Control']
for i in range(3):
    ax = axes[0, i]
    
    phi1_mask = ((phi1.value > phi1_lims[i][0]) & 
                 (phi1.value < phi1_lims[i][1]) &
                 (phi2.value > -5) & (phi2.value < 5))
    
    ax.hist(phi2[phi1_mask],
            bins=bins, alpha=1, 
            normed=True, color='#aaaaaa');
    
    sampler = all_samplers[i]
    idx = sampler.lnprobability[:, -1].argmax()
    show_p = sampler.chain[idx, -1]

    ax.plot(phi2_grid, np.exp(lnlike(show_p, phi2_grid)),
            lw=2, marker='', color='k', zorder=100, alpha=0.8,
            label='MAP density')
    ax.set_xlabel(r'$\phi_2$ [deg]')
    
    ax.set_title(titles[i])
    
#     if i == 0:
#         ax.legend(fontsize=16)
    
colors = ['#777777', 'k', '#bbbbbb']
zorders = [10, 5, 1]
for i in range(3):
    ax = axes[1, i]
    
    sampler = all_samplers[i]
    flatchain = np.vstack(sampler.chain[:, 2048::16])
    
    a_s1, a_s2, a_f, *_ = flatchain.T
    a_bg = 1 - (a_s1+a_s2+a_f)
    fs = [a_s1+a_s2, a_f, a_bg]
    names = [r'$\alpha_{\rm s}$', r'$\alpha_{\rm f}$', r'$\alpha_{\rm bg}$']
    
    print(np.percentile(a_f, [1, 99, 15, 85]))

    for f, name, color, zo in zip(fs, names, colors, zorders):
        ax.hist(f, bins=a_bins, histtype='step', 
                label=name, lw=2, color=color, zorder=zo, normed=True)
    
    if i == 0:
        ax.legend(fontsize=16)
        
    ax.set_xlabel(r'$\alpha$')
    
axes[0, 0].set_ylabel(r'density [${\rm deg}^{-1}$]')
axes[1, 0].set_ylabel(r'$p(\alpha)$')

# fig.set_facecolor('w')

fig.savefig('../paper/density-model.pdf')

In [ ]:
for i in range(3):
    sampler = all_samplers[i]
    flatchain = np.vstack(sampler.chain[:, 2048::16])
    
    a_s1, a_s2, a_f, *_ = flatchain.T
    a_bg = 1 - (a_s1+a_s2+a_f)
    
    print(np.percentile(a_f, [15, 85]))